In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv("~/Downloads/train.csv/train.csv")


In [30]:
data = np.array(data) # Convert data to array with numpy
m, n = data.shape # Dimensions of the data. m = number of entries, n = number of pixels
np.random.shuffle(data) # Shuffle data before splitting into training and dev sets

data_dev = data[0:1000].T # uses first 1000 examples for dev sets and transposes it
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T # uses the rest as training data
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_, m_train = X_train.shape

In [31]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forw_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [32]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forw_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [41]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 1000)

Iteration:  0
[6 6 6 ... 6 6 6] [0 8 4 ... 1 2 0]
0.0773170731707317
Iteration:  10
[6 9 6 ... 1 6 6] [0 8 4 ... 1 2 0]
0.14521951219512194
Iteration:  20
[0 9 6 ... 1 6 4] [0 8 4 ... 1 2 0]
0.2426829268292683
Iteration:  30
[0 9 6 ... 1 6 0] [0 8 4 ... 1 2 0]
0.3047560975609756
Iteration:  40
[0 9 4 ... 1 6 0] [0 8 4 ... 1 2 0]
0.3474634146341463
Iteration:  50
[0 9 4 ... 1 6 0] [0 8 4 ... 1 2 0]
0.38621951219512196
Iteration:  60
[0 9 4 ... 1 6 0] [0 8 4 ... 1 2 0]
0.43360975609756097
Iteration:  70
[0 9 4 ... 1 6 0] [0 8 4 ... 1 2 0]
0.49585365853658536
Iteration:  80
[0 8 4 ... 1 6 0] [0 8 4 ... 1 2 0]
0.5520975609756098
Iteration:  90
[0 8 4 ... 1 2 0] [0 8 4 ... 1 2 0]
0.5972439024390244
Iteration:  100
[0 8 4 ... 1 2 0] [0 8 4 ... 1 2 0]
0.6302682926829268
Iteration:  110
[0 8 4 ... 1 3 0] [0 8 4 ... 1 2 0]
0.6555853658536586
Iteration:  120
[0 8 4 ... 1 3 0] [0 8 4 ... 1 2 0]
0.6763170731707318
Iteration:  130
[0 8 4 ... 1 3 0] [0 8 4 ... 1 2 0]
0.6939756097560975
Iteration:  1

In [42]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forw_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [43]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[8 4 7 5 1 7 4 7 9 4 8 4 0 8 8 7 7 0 6 2 8 0 2 5 8 3 5 6 5 2 6 3 0 6 9 5 4
 9 5 0 8 7 7 0 2 2 8 9 4 2 1 6 8 3 5 2 8 1 8 4 7 3 3 6 8 2 1 1 8 6 0 8 9 0
 1 0 5 1 3 2 6 1 2 5 3 2 9 7 0 0 0 7 3 6 7 9 6 2 1 1 8 0 8 2 9 8 9 9 0 1 8
 8 8 0 5 7 2 7 0 4 0 7 9 2 3 3 1 0 7 8 1 5 6 1 4 5 8 9 4 7 3 9 4 2 8 3 0 3
 9 8 4 2 8 3 4 0 7 3 8 2 1 7 3 3 1 2 9 9 4 2 0 7 3 3 1 8 4 2 4 9 7 6 1 1 5
 1 7 2 0 2 1 8 2 8 3 6 3 2 3 0 4 1 3 1 7 1 3 5 5 8 9 9 6 7 9 0 5 3 3 1 9 2
 9 5 9 6 4 7 9 0 9 8 8 3 0 5 9 7 5 2 4 7 7 2 1 1 3 5 9 7 6 6 7 2 2 2 4 8 8
 5 3 4 3 0 4 0 6 3 1 3 6 2 6 5 3 7 6 8 3 6 5 5 3 5 2 0 2 9 9 2 7 0 9 4 2 0
 9 2 8 8 0 3 6 2 1 0 5 8 2 0 8 7 0 0 6 3 6 8 7 2 4 7 7 0 9 1 8 2 8 6 0 4 0
 0 7 1 0 0 0 2 3 5 3 6 4 4 1 2 8 2 5 1 9 0 5 6 1 4 1 8 6 6 3 6 2 4 7 2 4 2
 1 1 3 8 6 9 7 8 0 7 9 4 0 9 7 1 8 7 3 1 0 0 0 8 9 9 9 2 2 4 6 6 1 0 1 4 2
 6 3 5 6 8 1 2 0 6 6 0 7 0 0 9 4 5 7 9 4 3 7 4 8 0 9 4 9 7 8 9 1 7 8 0 4 9
 6 8 2 6 0 8 5 3 6 2 7 3 7 8 4 0 9 5 3 4 1 4 6 1 0 9 9 9 8 0 5 6 5 6 2 4 9
 4 1 3 0 1 3 0 7 1 6 9 5 

np.float64(0.875)